In [1]:
# All stores in Dallas DMA for Smoothie King
# All stores in Dallas DMA for Big Lots & All stores in the largets DMA ofr Big Lots

In [2]:
import pandas as pd
import zipcodes
import numpy as np
import datetime

In [3]:
Zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
Zip_DMA=Zip_DMA.iloc[:,[0,2]]
Zip_DMA.columns=['zip_cd','DMA']
Zip_DMA=Zip_DMA.drop_duplicates()
Zip_DMA_DALLAS=Zip_DMA[Zip_DMA['DMA']=="DALLAS-FT. WORTH"]

In [4]:
SK_revised_store_zip=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_2018-10-31.xlsx",dtype=str,sheetname="stores_zips_overwritten")
SK_revised_store_zip=SK_revised_store_zip[['storenumber','revised_store_zip']]
SK_revised_store_zip['storenumber']=SK_revised_store_zip['storenumber'].astype(int).astype(str)
SK_revised_store_zip_dict=SK_revised_store_zip.set_index(['storenumber']).to_dict()['revised_store_zip']

In [5]:
SK_stores=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/StoreList.xlsx",dtype=str)
SK_stores=SK_stores[SK_stores['status']!="Closed"]
SK_stores['storenumber']=SK_stores['storenumber'].astype(int).astype(str)
SK_stores=SK_stores.reset_index()
del SK_stores['index']
SK_stores['revised_store_zip']=np.nan

for i in range(len(SK_stores)):
    if SK_stores['storenumber'][i] in SK_revised_store_zip_dict.keys():
        SK_stores['revised_store_zip'][i]=SK_revised_store_zip_dict[SK_stores['storenumber'][i]]
    else:
        SK_stores['revised_store_zip'][i]=SK_stores['zip'][i].split("-")[0].zfill(5)
SK_stores['revised_store_zip']=SK_stores['revised_store_zip'].astype(int).astype(str).apply(lambda x: x.zfill(5))
SK_stores['zip_cd']=SK_stores['zip'].apply(lambda x: x.split("-")[0].zfill(5))
SK_stores_DALLAS=SK_stores[(SK_stores['zip_cd'].isin(Zip_DMA_DALLAS['zip_cd'])) | (SK_stores['revised_store_zip'].isin(Zip_DMA_DALLAS['zip_cd']))]
SK_stores_DALLAS=pd.merge(SK_stores_DALLAS,Zip_DMA_DALLAS,on="zip_cd",how="left")


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
BL_store=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181101-134628-331.txt",dtype=str,sep="|")
sales_20181027=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-10-27.csv",dtype=str)
sales_20181027=sales_20181027[sales_20181027['week_end_date']=="2018-10-27"]
sales_20181027['sales']=sales_20181027['sales'].astype(float)
sales_20181027=sales_20181027[sales_20181027['sales']>0]
BL_store=BL_store[BL_store['location_id'].isin(sales_20181027['location_id'])]
BL_store['zip_cd']=BL_store['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

In [7]:
BL_store=pd.merge(BL_store,Zip_DMA,on="zip_cd",how="left")
BL_store_count=BL_store.groupby(['DMA'])['location_id'].count().to_frame().reset_index().sort_values(['location_id'],ascending=False)
BL_store_count.head(2)
BL_store_LA=BL_store[BL_store['DMA']==BL_store_count['DMA'].tolist()[0]]
BL_store_DALLAS=BL_store[BL_store['DMA']==BL_store_count['DMA'].tolist()[1]]

In [8]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/SK_BL_Store_Dallas_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
SK_stores_DALLAS.to_excel(writer,"SK_Store_Dallas",index=False)
BL_store_LA.to_excel(writer,"BL_Store_LosAngeles",index=False)
BL_store_DALLAS.to_excel(writer,"BL_Store_Dallas",index=False)
writer.save()